In [1]:
# Import libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# Load models
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-spanish-cased-finetuned-ner")
model = AutoModelForTokenClassification.from_pretrained("mrm8488/bert-spanish-cased-finetuned-ner")

/home/fxr/.local/share/virtualenvs/ner_spanish-HXC9jEQ8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Read 'csv' file as dataframe
df = pd.read_csv('../data/raw/nicaragua.csv')

# Create a copy of dataframe
df_copy = df.copy()

# Show dataframe
df_copy

,Fecha,tipo,corregido
0,2022/05/24,otras nacionalidades,"1. Fundación Terre Des Hommes Italia-Onlus, or..."
1,2022/05/24,otras nacionalidades,"2. Asociación Vivamos Mejor, originario de Cos..."
2,2022/05/24,otras nacionalidades,"3. Médico Internacional E.V., originario de Al..."
3,2022/05/24,otras nacionalidades,"4. Planting Hope, INC (Sembrando Esperanza, IN..."
4,2022/05/24,otras nacionalidades,"5. Proyecto Gettysburg-León, INC, originario d..."
...,...,...,...
1425,2022/12/09,otras nacionalidades,"5. Junta Misionera Internacional, originario d..."
1426,2022/12/09,otras nacionalidades,"6. Visedal Partnership, Inc, originario de Est..."
1427,2022/12/09,otras nacionalidades,"7. Baptist International Missions, Inc, ongmar..."
1428,2022/12/09,otras nacionalidades,8. Mellemfolkeligt Samvirke/ Asociacion Danesa...


In [8]:
# Create instance for classifier with NER
classifier = pipeline("ner", model=model, tokenizer=tokenizer)

In [9]:
# Convert column to list
text = df['corregido'].tolist()

In [10]:
# Analyze NER from list and extract ORG type
orgs = []
for i in tqdm(range(len(text))):
    doc = classifier(text[i])
    org = [token.get('word') for token in doc if token.get('entity') == 'I-ORG']
    orgs.append(org)

100%|██████████| 1430/1430 [03:23<00:00,  7.02it/s]


In [12]:
orgs

[['Ter', '##re', 'Des', 'Hom', '##mes', 'Italia', '##lus'],
 ['Viva', '##mos', 'Mejor'],
 [],
 ['##ting', 'Hope', 'IN', '##C', '##brando', 'Esperanza', '##C', 'Unidos'],
 ['##C', 'Unidos'],
 ['para', 'la', 'Cooperación', 'Internacional'],
 ['International', 'Unidos'],
 ['##mac', '##e', '##uti', '##cos', 'Mun', '##di'],
 ['Minnesota'],
 ['##ject', 'Cha', '##coc', '##ente', 'Unidos'],
 ['##n',
  'Human',
  '##itaria',
  '##n',
  'and',
  'Development',
  'Foundation',
  '##C',
  '##P',
  'Unidos'],
 ['SNVNederlandseOntwikkelingsorganisatie',
  'Hola',
  '##n',
  '##des',
  'de',
  'Coopera',
  '##cion',
  'al',
  'Desarrollo'],
 [],
 ['para',
  'el',
  'Auto',
  '##desarro',
  '##llo',
  ',',
  'el',
  'Intercambi',
  '##o',
  'y',
  'la',
  'Solidaridad'],
 ['Nuestros', 'Pequeño', '##s', 'Hermanos', 'Hermana', '##s', 'Internacional'],
 [],
 ['De', 'La', 'Mujer', 'Mar', '##jor', '##ie'],
 ['##acion', 'De', 'Co', '##uri', '##ers', 'De', 'Nicaragua'],
 ['Grupo', 'Campe', '##sin', '##o', 'F

In [13]:
# Create a new column from list
df_copy['organization'] = pd.Series(orgs)

# Replace empty lists with NaN values
# df.orgs = df.orgs.apply(lambda x: np.nan if len(x)==0 else x)

# Show dataframe
df_copy

,Fecha,tipo,corregido,organization
0,2022/05/24,otras nacionalidades,"1. Fundación Terre Des Hommes Italia-Onlus, or...","[Ter, ##re, Des, Hom, ##mes, Italia, ##lus]"
1,2022/05/24,otras nacionalidades,"2. Asociación Vivamos Mejor, originario de Cos...","[Viva, ##mos, Mejor]"
2,2022/05/24,otras nacionalidades,"3. Médico Internacional E.V., originario de Al...",[]
3,2022/05/24,otras nacionalidades,"4. Planting Hope, INC (Sembrando Esperanza, IN...","[##ting, Hope, IN, ##C, ##brando, Esperanza, #..."
4,2022/05/24,otras nacionalidades,"5. Proyecto Gettysburg-León, INC, originario d...","[##C, Unidos]"
...,...,...,...,...
1425,2022/12/09,otras nacionalidades,"5. Junta Misionera Internacional, originario d...","[Mis, ##ione, ##ra, Internacional, Unidos]"
1426,2022/12/09,otras nacionalidades,"6. Visedal Partnership, Inc, originario de Est...","[##eda, ##l, Part, ##ners, ##hip]"
1427,2022/12/09,otras nacionalidades,"7. Baptist International Missions, Inc, ongmar...","[., Ba, ##pti, ##st, International, Miss, ##io..."
1428,2022/12/09,otras nacionalidades,8. Mellemfolkeligt Samvirke/ Asociacion Danesa...,"[Dan, ##esa, para, la, Coopera, ##cion, Intern..."


In [ ]:
# Save dataframe as csv
df_copy.to_csv('../data/processed/orgs_bert.csv', index=False)